In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"UTC\"")

In [ ]:
hex_project_id = _hex_json.loads("\"3ee03fe6-6484-4a3a-bd91-a5e3969733ef\"")

In [ ]:
hex_project_name = _hex_json.loads("\"Forecasting Hourly Traffic\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

# Setup
Install required packages and make sure all the libraries we need have been imported.

In [ ]:
import hextoolkit
hex_snowflake_conn = hextoolkit.get_data_connection('[Demo] Hex Public Data')
session = hex_snowflake_conn.get_snowpark_session()

In [ ]:
install = _hex_json.loads("false")

In [ ]:
if install:
    !pip install mockseries
    !pip install --upgrade xgboost
else:
    print("Click the button to install the required packages")

INFO: pip is looking at multiple versions of mockseries to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 92.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 87.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 65.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.5
    Uninstalling matplotlib-3.7.5:
      Successfully uninstalled matplotlib-3.7.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [ ]:
# Mock Data
import warnings
warnings.filterwarnings('ignore')
from mockseries.trend import LinearTrend
from mockseries.seasonality import SinusoidalSeasonality
from mockseries.noise import RedNoise
from mockseries.utils import plot_timeseries, write_csv
from datetime import datetime, date, timedelta
from mockseries.utils import datetime_range
from pandas.tseries.holiday import USFederalHolidayCalendar
import numpy as np
import random

## Snowpark 
import snowflake.snowpark
from snowflake.snowpark.session import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import col
from snowflake.snowpark.types import StringType, DateType, TimestampType, IntegerType, StructType, StructField, FloatType
from snowflake.snowpark.functions import dateadd, current_date
from snowflake.snowpark.functions import udtf

# Model Deployment
from time import time
import sys, string, io, os, math
import zipfile, json, pickle
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Establish Secure Connection to Snowflake
Add the schema to use throughout this project to the snowpark session connection. This ensures that data is always written to the correct location.

# Creating the data



In [ ]:
class Dataset:
    ''' Used to generate the data used for forecasting '''
    def __init__(self):
        trend = LinearTrend(coefficient=0.025, time_unit=timedelta(days=4), flat_base=100)
        seasonality = SinusoidalSeasonality(
            amplitude=20, period=timedelta(days=7)
        ) + SinusoidalSeasonality(amplitude=4, period=timedelta(days=1))
        noise = RedNoise(mean=0, std=3, correlation=0.5)
        timeseries = trend + seasonality + noise

        self.time_points = datetime_range(
        granularity=timedelta(hours=1),
        start_time=datetime(2018, 6, 16),
        end_time=datetime.today())
        self.ts_values = timeseries.generate(time_points=self.time_points)
        self.data = None
        self.dataset = None
        self.calendar = None

    def plot(self):
        plot_timeseries(self.time_points, self.ts_values)

    def _create_dataframe(self):
        df = pd.DataFrame({"time_points":self.time_points,"ts_values":self.ts_values})
        df['hour'] = df['time_points'].dt.hour
        df['date'] = df['time_points'].dt.date
        df['dayofweek'] = df['time_points'].dt.weekday

        df = df[df['hour'].between(7,22)]

        df['weekday'] = np.where((df['dayofweek']>= 1) & (df['dayofweek']<= 4), 1, 0)
        df['weekend'] = np.where((df['dayofweek']>= 5) & (df['dayofweek']<= 6), 1, 0)
        df['sunday'] = np.where(df['dayofweek']== 0 , 1, 0)
        df['breakfast'] = np.where((df['hour']>= 7) & (df['hour']<= 10), 1, 0)
        df['lunch'] = np.where((df['hour']>= 11) & (df['hour']<= 13), 1, 0)
        df['break'] = np.where((df['hour']>= 14) & (df['hour']<= 15), 1, 0)
        df['dinner'] = np.where((df['hour']>= 16) & (df['hour']<= 20), 1, 0)
        df['close'] = np.where(df['hour']>= 21, 1, 0)
        self.data = df

    def _create_date_table(self, start='2018-01-01', end='2025-12-31'):
        df = pd.DataFrame({"CALENDAR_DATE": pd.date_range(start, end)})
        df["CALENDAR_WEEK_DAY_NBR"] = df.CALENDAR_DATE.dt.dayofweek
        df["CALENDAR_MTH_DAY_NBR"] = df.CALENDAR_DATE.dt.day
        df["CALENDAR_MTH"] = df.CALENDAR_DATE.dt.month
        df["CALENDAR_YEAR"] = df.CALENDAR_DATE.dt.year
        return df
    
    def _random_traffic(self, r):
        if r["close"] == 1:
            return random.uniform(0,.1)
        elif (r["weekday"] == 1 and r["break"] == 1) or (r["weekday"] == 1 and r["breakfast"] == 1):
            return random.uniform(.15,.2)
        elif (r["weekday"] == 1 and r["dinner"] == 1) or (r["weekend"] == 1 and r["break"] == 1):
            return random.uniform(.25,.35)
        elif (r["sunday"] == 1) and (r["dinner"] == 1):
            return random.uniform(.4,.5)
        elif (r["weekend"] == 1 and r["breakfast"] == 1) or (r["weekend"] == 1 and r["dinner"] == 1) or (r["weekday"] == 1 and r["lunch"] == 1) or (r["sunday"] == 1 and r["break"] == 1):
            return random.uniform(.52,.65)
        elif (r["sunday"] == 1 and r["breakfast"] == 1) or (r["weekend"] == 1 and r["lunch"] == 1):
            return random.uniform(.70,.8)
        elif (r["sunday"] == 1 and r["lunch"] == 1):
            return random.uniform(.95,1)
        else:
            return 0

    def create_traffic_table(self):
        ''' Run this function to simulate the hourly traffic which is returned as a snowflake dataframe'''

        print('Creating Initial Dataframe...', end = " ")
        self._create_dataframe()
        print("Complete!")

        print('Simulating restaurant traffic... (about a 3 min wait)', end = ' ')
        dfs = []
        # takes abt 3 min to run
        for i in range (1,201):
            _ = self.data.copy()
            _['store_id'] = i
            _['college_town'] = np.random.randint(0,2)
            _["rest_shift"] = self.data.apply(self._random_traffic, axis = 1)
            dfs.append(_)

        self.data = pd.concat(dfs)
        print("Complete!")
    
        print("Adding US calendar Holidays...", end = " ")
        calendar = USFederalHolidayCalendar()
        holiday_df = (
            pd.DataFrame(
                calendar.holidays(start=min(self.data["date"]), end=max(self.data["date"]), return_name=True)
            )
            .reset_index()
            .rename(columns={"index": "date", 0: "holiday_name"})
        )

        holiday_df['date'] = holiday_df['date'].dt.date
        self.data = self.data.merge(holiday_df, on = 'date', how = 'left')
        self.data['hourly_traffic'] = self.data.ts_values * self.data.rest_shift
        print("Complete!")

        print("Creating master dataset...", end = " ")
        final = self.data[['time_points', 'hourly_traffic','holiday_name','store_id','college_town']]
        final['hourly_traffic'] = pd.to_numeric(final['hourly_traffic'])
        final['hourly_traffic'] = final['hourly_traffic'].astype(float)
        final['hourly_traffic'] = final['hourly_traffic'].round()
        final['hourly_traffic'] = final['hourly_traffic'].astype(int)
        final = final.rename(columns={"time_points": "TIME_POINTS", "hourly_traffic": "HOURLY_TRAFFIC","holiday_name": "HOLIDAY_NAME","store_id": "STORE_ID","college_town":"COLLEGE_TOWN"})
        self.dataset = final  
        print("Master dataset created!\nWrite this dataframe back into your Snowflake database.")
        return self.dataset

    def create_calendar_table(self, session):
        print("Creating calendar...", end = " ")
        calendar_df = self._create_date_table()

        calendar = USFederalHolidayCalendar()
        holiday_df = (
            pd.DataFrame(
                calendar.holidays(start='2018-01-01', end='2025-12-31', return_name=True)
            )
            .reset_index()
            .rename(columns={"index": "date", 0: "holiday_name"})
        )

        holiday_df['date'] = holiday_df['date'].dt.date
        calendar_df['CALENDAR_DATE'] = calendar_df['CALENDAR_DATE'].dt.date
        calendar_final = calendar_df.merge(holiday_df, left_on='CALENDAR_DATE', right_on='date', how = 'left')
        calendar_final = calendar_final.rename(columns={"holiday_name":"HOLIDAY_NAME"})
        calendar_final_snow_df = session.create_dataframe(calendar_final).select('CALENDAR_DATE','CALENDAR_WEEK_DAY_NBR','CALENDAR_MTH_DAY_NBR','CALENDAR_MTH','CALENDAR_YEAR','HOLIDAY_NAME')
        calendar_final_snow_df = calendar_final_snow_df.select(
            col("CALENDAR_DATE"),
            col("CALENDAR_WEEK_DAY_NBR").cast(StringType()).alias("CALENDAR_WEEK_DAY_NBR"),
            col("CALENDAR_MTH_DAY_NBR").cast(StringType()).alias("CALENDAR_MTH_DAY_NBR"),
            col("CALENDAR_MTH").cast(StringType()).alias("CALENDAR_MTH"),
            col("CALENDAR_YEAR").cast(StringType()).alias("CALENDAR_YEAR"),
            col("HOLIDAY_NAME"),
        )
        self.calendar = calendar_final_snow_df.toPandas()
        print("Complete!")

        return self.calendar

In [ ]:
# create an instance of the dataset class so that we can get our hourly traffic dataset
dataset = Dataset()

In [ ]:
# We can get a pandas dataframe by calling this function, which generates the data for us
traffic = dataset.create_traffic_table()

Creating Initial Dataframe... Complete!
Simulating restaurant traffic... (about a 3 min wait) Complete!
Adding US calendar Holidays... Complete!
Creating master dataset... Master dataset created!
Write this dataframe back into your Snowflake database.


In [ ]:
cal = dataset.create_calendar_table(session)

Creating calendar... 

## Write data back to database



In [ ]:
#Add stage for UDFs and Stored Procs
session.sql('''
create stage if not exists pymodels
''').collect()

# Explore Historical Data

Let's look at the historical **HOURLY_TRAFFIC** and **CALENDAR_INFO** tables from the stores.

In [ ]:
store_hourly_info_df = session.table('HOURLY_TRAFFIC')
store_hourly_info_df.limit(10).toPandas()

In [ ]:
store_calendar_info_df = session.table('CALENDAR_INFO')
store_calendar_info_df.limit(10).toPandas()

# Feature Engineering and Data Pre-Processing Tasks
We're going to create a feature table with past historical data and future data to pass in to our XGBoost model.

### Create a dataframe with relevant historical data

In [ ]:
## Extract date and hour from the time stamp in Hourly traffic
past = store_hourly_info_df.select(
    "TIME_POINTS",
    col("TIME_POINTS").cast(DateType()).alias("DATE"),
    F.hour(col("TIME_POINTS")).alias("HOUR"),
    "STORE_ID",
    "COLLEGE_TOWN",
    "HOURLY_TRAFFIC",
)


## Join the Calendar info table to the Hourly traffic table
## Filter hour between 7 and 22 since the restaraunts are only open from 7am -> 10pm
past_final = (
    past.join(
        store_calendar_info_df,
        (store_calendar_info_df.col("CALENDAR_DATE") == past.col("DATE")),
        "left",
    )
    .select(
        col("TIME_POINTS"),
        col("HOUR"),
        "STORE_ID",
        "COLLEGE_TOWN",
        "CALENDAR_WEEK_DAY_NBR",
        "CALENDAR_MTH_DAY_NBR",
        "CALENDAR_MTH",
        "CALENDAR_YEAR",
        "HOLIDAY_NAME",
        "HOURLY_TRAFFIC",
    )
    .filter(col("HOUR").between(7, 22))
    .na.fill({"HOLIDAY_NAME": "No Holiday"})
)

past_final.limit(5).toPandas()

### Generate store info with a empty hourly traffic for the next four weeks for forecasting 

In [ ]:
## Create a column that has the next 672 hours (28 days) in date time format.
df_date = session.range(672).select(dateadd("HOUR", "ID", current_date()).as_("DATE"))

df_date = df_date.with_column("HOUR",F.hour(df_date["DATE"]))

df_date = df_date.select(F.to_date(df_date["DATE"]).as_("DATE"),'HOUR').filter(col('HOUR').between(7,22))

 ## Cross join to make sure each store gets a value for the next 4 weeks
df_store = session.table('HOURLY_TRAFFIC').select(col('STORE_ID').cast("string").alias("STORE_ID"),col('COLLEGE_TOWN').cast("string").alias("COLLEGE_TOWN")).distinct()
stores = df_date.cross_join(df_store)

In [ ]:
## Add in Calendar Information to create the final future table
future_cal = session.table('CALENDAR_INFO')\
    .select('CALENDAR_DATE',\
            'CALENDAR_WEEK_DAY_NBR',\
            'CALENDAR_MTH_DAY_NBR',\
            'CALENDAR_MTH',\
            'CALENDAR_YEAR',\
            'HOLIDAY_NAME').\
    filter((F.col('CALENDAR_DATE') >= F.current_date())\
                                          & (F.col('CALENDAR_DATE')  <= F.current_date()+28))

future_cal = future_cal.na.fill({"HOLIDAY_NAME": 'No Holiday'})

## Join store info and calendar data
future_df = stores.join(future_cal, stores.col("DATE") == future_cal.col("CALENDAR_DATE"),"right")
future_df = future_df.drop('CALENDAR_DATE')

future_df = future_df.withColumn("DATE_HOUR", F.to_timestamp(F.dateadd("hour",col("HOUR"),col("DATE"))))
future_df = future_df.drop('DATE')

future_df = future_df.withColumn('HOURLY_TRAFFIC', F.lit(0))

future_df = future_df.select('DATE_HOUR',\
                'HOUR',\
                'STORE_ID',\
                'COLLEGE_TOWN',\
                'CALENDAR_WEEK_DAY_NBR',\
                'CALENDAR_MTH_DAY_NBR',\
                'CALENDAR_MTH',\
                'CALENDAR_YEAR',\
                'HOLIDAY_NAME',\
                'HOURLY_TRAFFIC')

future_df.limit(5).toPandas()

### Save final features to a new table 

In [ ]:
 ## Union the historical and future tables together
unionDF = past_final.union(future_df)
    
## Write the final features table to Snowflake 
unionDF.write.saveAsTable('MODEL_FEATURES', mode='overwrite', create_temp_table=False)

## Training & batch forecasts using a UDTFs. 

In [ ]:
schema = StructType([
     StructField("DATE", DateType()),
    StructField("HOUR_OF_DAY", IntegerType()),
    StructField("HOURLY_FORECAST", FloatType())  
])

In [ ]:
@udtf(output_schema = schema,
     input_types = [TimestampType(), IntegerType(),IntegerType(),FloatType(),StringType(),StringType(),StringType(),StringType()],
     name = "store_forecast", is_permanent=True, stage_location="@pymodels",
     packages=["pandas","xgboost == 1.5.0"], replace=True, session=session)
class forecast:
    def __init__(self):
        self.date_hour=[]
        self.from_hour=[]
        self.COLLEGE_TOWN=[]
        self.DAYOFWEEK=[]
        self.MONTH=[]
        self.YEAR=[]
        self.HOLIDAY_NAME=[]
        self.HOURLY_TRAFFIC=[]
    
    def process(self, date_hour, HOURLY_TRAFFIC, from_hour, COLLEGE_TOWN, DAYOFWEEK, MONTH, YEAR, HOLIDAY_NAME):
        self.date_hour.append(date_hour)
        self.HOURLY_TRAFFIC.append(HOURLY_TRAFFIC)
        self.from_hour.append(from_hour)
        self.COLLEGE_TOWN.append(COLLEGE_TOWN)
        self.DAYOFWEEK.append(DAYOFWEEK)
        self.MONTH.append(MONTH)
        self.YEAR.append(YEAR)
        self.HOLIDAY_NAME.append(HOLIDAY_NAME)
    
    def end_partition(self):
        df = pd.DataFrame(zip(self.date_hour, 
                              self.HOURLY_TRAFFIC, 
                              self.from_hour, 
                              self.COLLEGE_TOWN, 
                              self.DAYOFWEEK, 
                              self.MONTH, 
                              self.YEAR, 
                              self.HOLIDAY_NAME), 
                          columns = ['DATE_HOUR','HOURLY_TRAFFIC','HOUR','COLLEGE_TOWN','CALENDAR_WEEK_DAY_NBR',
                                     'CALENDAR_MTH','CALENDAR_YEAR','HOLIDAY_NAME'])
        
        # set the time column as our index 
        df2 = df.set_index('DATE_HOUR') 
        df2.index = pd.to_datetime(df2.index)

         # Converting features to categories for get_dummies
        df2['CALENDAR_WEEK_DAY_NBR'] = df2['CALENDAR_WEEK_DAY_NBR'].astype("category")
        df2['CALENDAR_MTH'] = df2['CALENDAR_MTH'].astype("category")
        df2['CALENDAR_YEAR'] = df2['CALENDAR_YEAR'].astype("category")
        df2['HOUR'] = df2['HOUR'].astype("category")
        df2['HOLIDAY_NAME'] = df2['HOLIDAY_NAME'].astype("category")
        df2['COLLEGE_TOWN'] = df2['COLLEGE_TOWN'].astype("category")

        #Use get_dummies for categorical features
        final = pd.get_dummies(data=df2, columns=['HOLIDAY_NAME', 
                                                  'COLLEGE_TOWN','CALENDAR_WEEK_DAY_NBR','CALENDAR_MTH','CALENDAR_YEAR','HOUR'])
       
        #do the train & forecast split
        today = date.today()
        yesterday = today - timedelta(days = 1)
        fourweek = today + timedelta(days = 28)
        tomorrow = today + timedelta(days = 1)

        train = final[(final.index >= pd.to_datetime('16-Jun-2018')) & (final.index <= pd.to_datetime(yesterday))]
        forecast = final[(final.index >= pd.to_datetime(tomorrow)) & (final.index <=pd.to_datetime(fourweek))]

        X_train = train.drop('HOURLY_TRAFFIC', axis = 1)
        y_train = train['HOURLY_TRAFFIC']

        X_forecast = forecast.drop('HOURLY_TRAFFIC', axis = 1)
        
        #Use XGBoost regressor model
        model = xgb.XGBRegressor(n_estimators=200,n_jobs=1)
        model.fit(X_train, y_train,
                verbose=False) 
        
        forecast['PREDICTION'] = model.predict(X_forecast)

        hours = forecast.index.hour
        forecast = pd.concat([forecast, pd.DataFrame(hours, index=forecast.index)], axis = 1)
        forecast = forecast[["DATE_HOUR","PREDICTION"]]
        forecast = forecast.sort_index()
        forecast.loc[forecast['PREDICTION'] < 0,'PREDICTION']=0
        forecast['DATE'] = forecast.index.date
        
        # output prediction
        for idx, row in forecast.iterrows():
            DATE = row['DATE']
            DATE_HOUR = row['DATE_HOUR']
            PREDICTION = row['PREDICTION']
            yield DATE, DATE_HOUR, PREDICTION

## Call the UDTF on Snowpark Optimized WH to run models in prallel and get forecast

In [ ]:
df = session.table("MODEL_FEATURES")
store_forecast = F.table_function("store_forecast")

In [ ]:
forecast = df.select(
    df["STORE_ID"],
    (
        store_forecast(
            df["TIME_POINTS"],
            df["HOURLY_TRAFFIC"],
            df["HOUR"],
            df["COLLEGE_TOWN"].cast(FloatType()),#.alias('COLLEGE_TOWN'),
            df["CALENDAR_WEEK_DAY_NBR"],
            df["CALENDAR_MTH_DAY_NBR"],
            df["CALENDAR_YEAR"],
            df["HOLIDAY_NAME"],
        ).over(partition_by=df["STORE_ID"])
    ),
)

In [ ]:
forecast.limit(10).show()

In [ ]:
# takes abt 7 minutes (on an x-small warehouse)
forecast.write.saveAsTable('FOUR_WEEK_FORECAST', mode='overwrite', create_temp_table=False)

In [ ]:
# import jinja2
# raw_query = """
#     select * from "PC_HEX_DB"."PUBLIC"."FOUR_WEEK_FORECAST"
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
date_hour = np.array(list(zip(cast['DATE'].astype('str'), cast["HOUR_OF_DAY"].astype('str'))))
full_time = []
for date, hour in date_hour:
    full_time.append(pd.to_datetime(f"{date} {hour}:00"))

cast['time'] = full_time

In [ ]:
import altair
chart_cast = altair.Chart.from_json("""
{
    "width": 500,
    "height": 500,
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "data": {
                "name": "layer00"
            },
            "mark": {
                "type": "line",
                "clip": true,
                "tooltip": true
            },
            "encoding": {
                "x": {
                    "field": "time",
                    "type": "temporal",
                    "timeUnit": "yearmonthdatehours"
                },
                "y": {
                    "field": "HOURLY_FORECAST",
                    "type": "quantitative"
                }
            }
        }
    ],
    "resolve": {
        "scale": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    }
}
""")
chart_cast.datasets.layer00 = cast.to_json(orient='records')
chart_cast.display(actions=False)